# Part 3c - Building a flair detector

In [3]:
# Get the required libraries and functions

import numpy as np
import pandas as pd
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding, Layer, Dense, Input, LSTM, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.initializers import Constant
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint
import warnings
warnings.filterwarnings('ignore') 

Using TensorFlow backend.


In [0]:
# Prepare the data
df = pd.read_csv('balanced_data.csv')

# Tokenize the posts, ie assign keys to each of the words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.title)
X = tokenizer.texts_to_sequences(df.title)
df['tokenized'] = X
X = list(sequence.pad_sequences(df.tokenized, maxlen=50))
word_index = tokenizer.word_index

In [0]:
# Convert the flairs into keys, and get an inverse mapping as well

flairs = df.groupby('flair').size().index.tolist()
flair_to_key = {}
key_to_flair = {}
for key, flair in enumerate(flairs):
  flair_to_key[flair] = key
  key_to_flair[key] = flair

df['key'] = df['flair'].apply(lambda x: flair_to_key[x])

In [0]:
# Prepare train and validation sets

X = np.array(X)
Y = to_categorical(list(df.key))
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.15, stratify=Y)

## Attention BiLSTM
We use an Attention BiLSTM architecture. The attention mechanism is as an improvement over the encoder decoder-based neural machine translation system in NLP. More details here: https://www.analyticsvidhya.com/blog/2019/11/comprehensive-guide-attention-mechanism-deep-learning/

In [0]:
# Building a standard Attention layer

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None
        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim
        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))
        if self.bias:
            eij += self.b
        eij = K.tanh(eij)
        a = K.exp(eij)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [ ]:
# Preparing the embedding layer using Stanford's 100 dimensional GloVe word vectors: https://nlp.stanford.edu/projects/glove/
# First we create a mapping from the words to their vector representation

mapping = {}
f = open('/content/gdrive/My Drive/glove.6B.100d.txt')
for embedding in f:
    embedding = embedding.split()
    word = embedding[0]
    vector = embedding[1:]
    vector = np.asarray(vector, dtype='float32')
    mapping[word] = vector
f.close()

In [0]:
# Now we create the actual embedding layer, and initialise it with the the veectors extracted
# We also freeze the layer, ie make it non-trainable

embedding_weights = np.zeros((len(word_index)+1, 100))
for word, idx in word_index.items():
    vector = mapping.get(word)
    if vector is not None:
        embedding_weights[idx] = vector

embedding_layer = Embedding(len(word_index)+1, 100, embeddings_initializer=Constant(embedding_weights), input_length=50, trainable=False)

In [10]:
# Preparing the model

inputs = Input(shape=(50,), dtype='int32')
x = embedding_layer(inputs)
x = Bidirectional(LSTM(512, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))(x)
x = Dropout(0.3)(x)
x = Attention(50)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
outputs = Dense(14, activation='softmax')(x)

checkpoint = ModelCheckpoint('best_weights.hdf5', monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='max')
model = Model(inputs, outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
history = model.fit(x_train, y_train, batch_size=64, epochs=15, validation_data=(x_val, y_val), callbacks=[checkpoint])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 25695 samples, validate on 4535 samples
Epoch 1/15
25664/25695 [============================>.] - ETA: 0s - loss: 2.0660 - acc: 0.3398
Epoch 00001: val_acc improved from -inf to 0.39846, saving model to best_weights.hdf5
25695/25695 [==============================] - 72s 3ms/sample - loss: 2.0657 - acc: 0.3398 - val_loss: 1.9324 - val_acc: 0.3985
Epoch 2/15
25664/25695 [============================>.] - ETA: 0s - loss: 1.8259 - acc: 0.4116
Epoch 00002: val_acc improved from 0.39846 to 0.45667, saving model to best_weights.hdf5
25695/25695 [==============================] - 69s 3ms/sample - loss: 1.8263 - acc: 0.4117 - val_loss: 1.6824 - val_acc: 0.4567
Epoch 3/15
25664/25695 [============================>.] - ETA: 0s - loss: 1.7124 - acc: 0.4501
Epoch 00003: val_acc improved from 0.45667 to 0.48291, saving model to best_weights.hdf5
25695/25695 [==============================] - 69s 3

In [0]:
# Saving the tokenizer

import pickle
pickle_out = open("tokenizer.pickle", "wb")
pickle.dump(tokenizer, pickle_out)
pickle_out.close()

## Testing

In [0]:
model.load_weights("best_weights.hdf5")

In [14]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

preds = model.predict(x_val)
y = np.asarray([np.argmax(line) for line in y_val])
preds = np.asarray([np.argmax(line) for line in preds])

print("Accuracy = {}".format(accuracy_score(y, preds)))
print("F1 score = {}".format(f1_score(y, preds, average='macro')))
print("Precision = {}".format(precision_score(y, preds, average='macro')))
print("Recall = {}".format(recall_score(y, preds, average='macro')))

Accuracy = 0.5570011025358325
F1 score = 0.5686316903518523
Precision = 0.576090558305631
Recall = 0.5766126358008499
